In [2]:
from Commutation import *


Sz = [Operator('S%d'%i, 'S^z_{%d}'%i) for i in [6, 1,2,3,4,5]]

a,b,c,d = [Operator('%s'%i, 'S^z_{%s}'%i) for i in 'abcd']



a_up = Operator('a⁺', 'S^+_{a}')
a_dn = Operator('a⁻', 'S^-_{a}')

b_up = Operator('b⁺', 'S^+_{b}')
b_dn = Operator('b⁻', 'S^-_{b}')

ca = CommutatorAlgebra()

def define_spinalg(plus, minus, z):
    ca.set_commutator(plus, minus)(2*z)
    ca.set_commutator(z, plus)( plus )
    ca.set_commutator(z, minus)( -1*minus )

In [3]:
define_spinalg(a_up, a_dn, a)
define_spinalg(b_up, b_dn, b)

for o in [a,b,c,d] + Sz:
    ca.add_operator(o)
# effectively scalars, idk why I bother

In [22]:
# define the Z objects

bond_zetas = [Operator('ζ%d'%i,r'\zeta_%d'%i,scalar=True) for i in range(1,4)]
bond_zetas_bar = [Operator('ζb%d'%i,r'\zeta_%d^*'%i,scalar=True) for i in range(1,4)]

def zeta(i):
    return bond_zetas[(i-1)%3]
def zeta_bar(i):
    return bond_zetas_bar[(i-1)%3]

KA = Operator('KA','K_A',scalar=True)
KB = Operator('KB','K_B',scalar=True)

def conjugate_zetas(x):
    repl = []
    for i in range(3):
        repl.append((bond_zetas[i], bond_zetas_bar[i]))
        repl.append((bond_zetas_bar[i], bond_zetas[i]))
    return x.replaceall(*repl)


def order_scalars(x):
    desired_order = [KA, KB] + bond_zetas + bond_zetas_bar + Sz + [a,b,c,d]
    for o in reversed(desired_order):
        ca.move_left(x, o)

# The setup: a and b are nearest-neighbour spins. They share a common tetrahedron,
# [abcd] which is $A$ sublattice.
# The two end tetrahedrons are $B$ sublattice, [123a] and [450b].
Za = KB*(zeta(1)*Sz[1] + zeta(2)*Sz[2] + zeta(3)*Sz[3]) + KA*(zeta(1)*b + zeta(2)* c + zeta(3)*d)
Zb = KB*(zeta(1)*Sz[4] + zeta(2)*Sz[5] + zeta(3)*Sz[0]) + KA*(zeta(1)*a + zeta(2)* d + zeta(3)*c)

Za_bar = conjugate_zetas(Za)
Zb_bar = conjugate_zetas(Zb)


hh = a_up*b_dn
kp = a_dn*Za_bar
km = b_up*Zb

hh_dag = a_dn*b_up
kp_dag = a_up*Za
km_dag = b_dn*Zb_bar

In [24]:
# Six possible operator orderings.
# Regardless of order, always have one [A + B]/2 and one [2A]/2 contrib,
# constant factors out. (It is 1/4(A+B)2A)
S_total =  hh*kp*km + kp*km*hh + km*hh*kp + hh*km*kp + km*kp*hh + kp*hh*km
S_total += hh_dag*kp_dag*km_dag + kp_dag*km_dag*hh_dag + km_dag*hh_dag*kp_dag + hh_dag*km_dag*kp_dag + km_dag*kp_dag*hh_dag + kp_dag*hh_dag*km_dag

order_scalars(S_total)

ca.move_right(S_total,a_up)
ca.move_right(S_total,a_dn)

ca.move_right(S_total,b_up)
ca.move_right(S_total,b_dn)


x1 = S_total.replaceall((a_up*a_dn, a+Fraction(1,2)),(b_up*b_dn, b+Fraction(1,2)))
x1.collect()


In [7]:
w = Operator('w','w',scalar=True)
w_bar = Operator('wb',r'w^*',scalar=True)

def simplify_zetas(expr):
    # only works if scalars are in canonical order
    simpl = []
    for i, z in enumerate(bond_zetas):
        for j, zb in enumerate(bond_zetas_bar):
            if i == j:
                simpl.append((z*zb, 1))
            elif i == (j+1) %3:
                simpl.append((z*zb, w))
            else:
                simpl.append((z*zb, w_bar))
    return expr.replaceall(*simpl).replaceall((w*w_bar, 1), (w_bar*w, 1))


x2 = simplify_zetas(x1)
order_scalars(x2)
x2.collect()
x2.show()
print(x2)

<IPython.core.display.Latex object>

  -4 KB KB S1 S4 a b  +3 KB KB S1 S4  +3/2 KB KB S1 S5 w  +3/2 KB KB S6 S1 wb  -4 KA KB S1 a a b  +1 KA KB S1 a  +3/2 KA KB S1 d w  +3/2 KA KB S1 c wb  +3/2 KB KB S2 S4 wb  -4 KB KB S2 S5 a b  +3 KB KB S2 S5  +3/2 KB KB S6 S2 w  +1/2 KA KB S2 a wb  -4 KA KB S2 a b d  +3 KA KB S2 d  +3/2 KA KB S2 c w  +3/2 KB KB S3 S4 w  +3/2 KB KB S3 S5 wb  -4 KB KB S6 S3 a b  +3 KB KB S6 S3  +1/2 KA KB S3 a w  +3/2 KA KB S3 d wb  -4 KA KB S3 a b c  +3 KA KB S3 c  -4 KA KB S4 a b b  +1 KA KB S4 b  +1/2 KA KB S5 b w  +1/2 KA KB S6 b wb  -4 KA KA a a b b  -1 KA KA a b  +3/2 KA KA b d w  +3/2 KA KA b c wb  +3/2 KA KB S4 c wb  -4 KA KB S5 a b c  +3 KA KB S5 c  +3/2 KA KB S6 c w  +3/2 KA KA a c wb  -8 KA KA a b c d  +6 KA KA c d  +3/2 KA KA c c w  +3/2 KA KB S4 d w  +3/2 KA KB S5 d wb  -4 KA KB S6 a b d  +3 KA KB S6 d  +3/2 KA KA a d w  +3/2 KA KA d d wb  -2 KB KB S1 S5 a b wb  +3/2 KB KB S1 S5 wb  -2 KB KB S6 S1 a b w  +3/2 KB KB S6 S1 w  -2 KA KB S1 a b d wb  +3/2 KA KB S1 d wb  -2 KA KB S1 a b c w  +3/2 

In [8]:
Zops = [a,b,c,d] + Sz


x3 = x2.replaceall( *[(z*z,Fraction(1,4)) for z in Zops])
x3.collect()
# x3.show()
print(x3)

<IPython.core.display.Latex object>

  -4 KB KB S1 S4 a b  +3 KB KB S1 S4  +3/2 KB KB S1 S5 w  +3/2 KB KB S6 S1 wb  +1 KA KB S1 b  +1 KA KB S1 a  +3/2 KA KB S1 d w  +3/2 KA KB S1 c wb  +3/2 KB KB S2 S4 wb  -4 KB KB S2 S5 a b  +3 KB KB S2 S5  +3/2 KB KB S6 S2 w  +1/2 KA KB S2 a wb  -4 KA KB S2 a b d  +3 KA KB S2 d  +3/2 KA KB S2 c w  +3/2 KB KB S3 S4 w  +3/2 KB KB S3 S5 wb  -4 KB KB S6 S3 a b  +3 KB KB S6 S3  +1/2 KA KB S3 a w  +3/2 KA KB S3 d wb  -4 KA KB S3 a b c  +3 KA KB S3 c  +1 KA KB S4 a  +1 KA KB S4 b  +1/2 KA KB S5 b w  +1/2 KA KB S6 b wb  +3/4 KA KA  -1 KA KA a b  +1 KA KA b d w  +1 KA KA b c wb  +3/2 KA KB S4 c wb  -4 KA KB S5 a b c  +3 KA KB S5 c  +3/2 KA KB S6 c w  +1 KA KA a c wb  -8 KA KA a b c d  +6 KA KA c d  +3/4 KA KA w  +3/2 KA KB S4 d w  +3/2 KA KB S5 d wb  -4 KA KB S6 a b d  +3 KA KB S6 d  +1 KA KA a d w  +3/4 KA KA wb  -2 KB KB S1 S5 a b wb  +3/2 KB KB S1 S5 wb  -2 KB KB S6 S1 a b w  +3/2 KB KB S6 S1 w  -2 KA KB S1 a b d wb  +3/2 KA KB S1 d wb  -2 KA KB S1 a b c w  +3/2 KA KB S1 c w  -2 KB KB S2 S4 a

In [9]:
root3 = Operator('√3',r'\sqrt{3}',scalar=True)

def convert_to_rectangular(expr):
    ca.move_right(expr, w)
    ca.move_right(expr, w_bar)
    coeff_w=expr.coefficient(w)
    coeff_wb=expr.coefficient(w_bar)
    
    remainder = expr + -1*(coeff_w*w +coeff_wb*w_bar)
    remainder.collect()
    real = remainder + Fraction(-1,2)*(coeff_w + coeff_wb)
    imag = Fraction(1,2)*(coeff_w + -1*coeff_wb)*root3
    return real, imag


x4, im = convert_to_rectangular(x3)
assert im == 0

In [8]:
x4.collect()
x4.show()

<IPython.core.display.Latex object>

In [13]:
# enforce an ice rule

x5 = x4.replaceall((a*b*c*d, Fraction(1,16)),(a*b*c, Fraction(1,4)*d), (a*b*d, Fraction(1,4)*c),(a*c*d, Fraction(1,4)*b),(b*c*d, Fraction(1,4)*a))
x5.collect()
x5.sort('multiplier')
x5.show()

<IPython.core.display.Latex object>

In [79]:
# enforce yet more ice rules
x6 = x5.replaceall( (Sz[1], -1*(Sz[3]+Sz[2]+a)), (Sz[4], -1*(Sz[0]+Sz[5]+b)))
order_scalars(x6)
x6 = x6.replaceall( (a*a, Fraction(1,4)), (b*b, Fraction(1,4)) )
x6.collect()

In [80]:
from itertools import permutations

quartic = Expression()
quad = Expression()

for t in x6.terms:
    if len(t) == 6:
        quartic += t
    else:
        quad += t
    
assert quad + quartic == x6

In [81]:
print(quad)

  -3/2 KB KB S3 a  -3/2 KB KB S2 a  -3/2 KB KB S6 b  -3/2 KB KB S5 b  -1/4 KB KB  -3/2 KA KB S2 c  -3/2 KA KB S3 d  -3/2 KA KB S5 d  -3/2 KA KB S6 c  +9/2 KB KB S3 S5  +9 KB KB S2 S5  +9/2 KB KB S5 a  +9 KB KB S6 S3  +9/2 KB KB S3 b  +9/2 KB KB S6 a  +9/2 KB KB S6 S2  +9/2 KB KB S2 b  +9/2 KA KB S2 d  +1 KA KB a d  -1 KA KA b d  +9/2 KA KB S6 d  +1 KA KB b d  -1 KA KA a d  +9/2 KA KB S3 c  +1 KA KB a c  -1 KA KA a c  -1 KA KA b c  +9/2 KA KB S5 c  +1 KA KB b c  -1/2 KA KA  -3/2 KA KB S3 a  -3/2 KA KB S5 b  -3/2 KA KB S2 b  -3/2 KA KB S2 a  -3/2 KA KB S6 a  -3/2 KA KB S6 b  -3/2 KA KB S5 a  -3/2 KA KB S3 b  -2 KA KB a b  -1/2 KA KB  +3 KB KB a b  +6 KA KA c d


In [86]:
# quartic.show()
y1, y2 = quartic.factor()
print(y1,"*", y2)

  -12 KB KB S6 S3  -6 KB KB S3 S5  -6 KB KB S6 S2  -12 KB KB S2 S5 * +1 a b
